In [1]:
import numpy as np
import pandas as pd

In [2]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2015.xlsx", encoding='utf8')

In [3]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,18h30,Sábado,09/05/2015,Chapecoense,Coritiba,2x1,Chapecoense,1ª Rodada,Arena Condá
1,18h30,Sábado,09/05/2015,Palmeiras,Atlético-MG,2x2,-,1ª Rodada,Allianz Parque
2,21h00,Sábado,09/05/2015,Fluminense,Joinville,1x0,Fluminense,1ª Rodada,Maracanã
3,11h00,Domingo,10/05/2015,Grêmio,Ponte Preta,3x3,-,1ª Rodada,Arena do Grêmio
4,16h00,Domingo,10/05/2015,Cruzeiro,Corinthians,0x1,Corinthians,1ª Rodada,Arena Pantanal


** Get all clubes for championship **



In [4]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [5]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [6]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,chapecoense
1,palmeiras
2,fluminense
3,grêmio
4,cruzeiro
5,atlético-pr
6,são paulo
7,sport
8,avaí
9,vasco


In [7]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [8]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,chapecoense,47.0,38.0,12.0,11.0,15.0,34.0,44.0,-10.0
1,palmeiras,53.0,38.0,15.0,8.0,15.0,60.0,51.0,9.0
2,fluminense,47.0,38.0,14.0,5.0,19.0,40.0,49.0,-9.0
3,grêmio,68.0,38.0,20.0,8.0,10.0,52.0,32.0,20.0
4,cruzeiro,55.0,38.0,15.0,10.0,13.0,44.0,35.0,9.0
5,atlético-pr,51.0,38.0,14.0,9.0,15.0,43.0,48.0,-5.0
6,são paulo,62.0,38.0,18.0,8.0,12.0,53.0,47.0,6.0
7,sport,59.0,38.0,15.0,14.0,9.0,53.0,38.0,15.0
8,avaí,42.0,38.0,11.0,9.0,18.0,38.0,60.0,-22.0
9,vasco,41.0,38.0,10.0,11.0,17.0,28.0,54.0,-26.0


In [9]:
reorderlist = [   "CORINTHIANS", "ATLÉTICO-MG", "GRÊMIO", "SÃO PAULO", "INTERNACIONAL", "SPORT", "SANTOS",
                  "CRUZEIRO", "PALMEIRAS", "ATLÉTICO-PR", "PONTE PRETA", "FLAMENGO", "FLUMINENSE",
                  "CHAPECOENSE", "CORITIBA", "FIGUEIRENSE", "AVAÍ", "VASCO", "GOIÁS", "JOINVILLE"
              ]
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,corinthians,81.0,38.0,24.0,9.0,5.0,71.0,31.0,40.0
2,atlético-mg,69.0,38.0,21.0,6.0,11.0,65.0,47.0,18.0
3,grêmio,68.0,38.0,20.0,8.0,10.0,52.0,32.0,20.0
4,são paulo,62.0,38.0,18.0,8.0,12.0,53.0,47.0,6.0
5,internacional,60.0,38.0,17.0,9.0,12.0,39.0,38.0,1.0
6,sport,59.0,38.0,15.0,14.0,9.0,53.0,38.0,15.0
7,santos,58.0,38.0,16.0,10.0,12.0,59.0,41.0,18.0
8,cruzeiro,55.0,38.0,15.0,10.0,13.0,44.0,35.0,9.0
9,palmeiras,53.0,38.0,15.0,8.0,15.0,60.0,51.0,9.0
10,atlético-pr,51.0,38.0,14.0,9.0,15.0,43.0,48.0,-5.0


** Por algum motivo que desconheço, na fonte dos dados o Vitória possui 39 pontos, na soma dos jogos, resulta em 38. **